In [1]:
from pipython import GCSDevice, pitools, datarectools
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import * #QDoubleValidator
import sys
import os
import pyvisa
import pickle
import numpy as np

class Window(QWidget):
    
    def __init__(self):
        super().__init__()
        self.title = 'GUI'
        self.left = 0
        self.top = 0
        self.width = 640
        self.height = 480
        
        
        
        self.stepSize_x = 0
        self.stepSize_y = 0
        self.stepSize_z = 0
        
        self.Xposition = 0
        self.Yposition = 0
        self.Zposition = 0
        self.screen = None
        
        self.power_value1 = 0
        self.power_value2 = 0

        self.measurement_index = 0
        
        self.initUI()
        
        
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)

        layout = QBoxLayout(QBoxLayout.LeftToRight)
        self.setLayout(layout)
        
        # Groupbox for Actuator
        groupbox = QGroupBox("X-Axis")
        layout.addWidget(groupbox)
        vbox = QVBoxLayout()
        groupbox.setLayout(vbox)


        # Button for forward X-axis
        button_forwardXAxis = QPushButton("Forward X-Axis")
        button_forwardXAxis.clicked.connect(self.XAxisForward)
        button_forwardXAxis.setToolTip("This button is used for moving X-axis in forward direction ")
        
        vbox.addWidget(button_forwardXAxis)

        # Button for backward X-axis
        button_backwardXAxis = QPushButton("Backward X-Axis")
        button_backwardXAxis.clicked.connect(self.XAxisBackward)
        button_backwardXAxis.setToolTip("This button is used for moving X-axis in backward direction ")
        vbox.addWidget(button_backwardXAxis)

        # Step size block
        self.lineedit = QLineEdit()
        self.lineedit.setPlaceholderText('Enter Step Size and Press Enter')
        self.lineedit.setValidator(QDoubleValidator())
        self.lineedit.returnPressed.connect(self.return_pressed_x)
        vbox.addWidget(self.lineedit)

        self.labelX = QLabel('X-Axis Value: 0', self)
        vbox.addWidget(self.labelX)


        # Group box for yth axis
        groupbox = QGroupBox("Y-axis")
        layout.addWidget(groupbox)
        vbox2 = QVBoxLayout()
        groupbox.setLayout(vbox2)

        # Button for forward Y-axis
        button_backwardYAxis = QPushButton("Forward Y-Axis")
        button_backwardYAxis.clicked.connect(self.YAxisBackward)
        button_backwardYAxis.setToolTip("This button is used for moving Y-axis in Forward direction ")
        vbox2.addWidget(button_backwardYAxis)

         # Button for backward Y-axis
        button_forwardYAxis = QPushButton("Backward Y-Axis")
        button_forwardYAxis.clicked.connect(self.YAxisForward)
        button_forwardYAxis.setToolTip("This button is used for moving Y-axis in Backward direction ")
        vbox2.addWidget(button_forwardYAxis)
        
        

        # Step size block For Y -axis
        self.lineedit1 = QLineEdit()
        self.lineedit1.setPlaceholderText('Enter Step Size and Press Enter')
        self.lineedit1.setValidator(QDoubleValidator())
        self.lineedit1.returnPressed.connect(self.return_pressed_y)
        vbox2.addWidget(self.lineedit1)

        self.labelY = QLabel('Y-Axis Value: 0', self)
        vbox2.addWidget(self.labelY)

        # Group box for zth axis
        groupbox = QGroupBox("Z-axis")
        layout.addWidget(groupbox)
        vbox3 = QVBoxLayout()
        groupbox.setLayout(vbox3)


         # Button for forward Z-axis
        button_forwardZAxis = QPushButton("Forward Z-Axis")
        button_forwardZAxis.clicked.connect(self.ZAxisForward)
        button_forwardZAxis.setToolTip("This button is used for moving Z-axis in forward direction ")
        vbox3.addWidget(button_forwardZAxis)
        
        # Button for backward Z-axis
        button_backwardZAxis = QPushButton("Backward Z-Axis")
        button_backwardZAxis.clicked.connect(self.ZAxisBackward)
        button_backwardZAxis.setToolTip("This button is used for moving Z-axis in backward direction ")
        vbox3.addWidget(button_backwardZAxis)

        # Step size block For Z -axis
        self.lineedit2 = QLineEdit()
        self.lineedit2.setPlaceholderText('Enter Step Size and Press Enter')
        self.lineedit2.setValidator(QDoubleValidator())
        self.lineedit2.returnPressed.connect(self.return_pressed_z)
        vbox3.addWidget(self.lineedit2)

        self.labelZ = QLabel('Z-Axis Value: 0', self)
        vbox3.addWidget(self.labelZ)

         # Group box for Sensor Measurement
        groupbox = QGroupBox("Sensor Measurement")
        layout.addWidget(groupbox)
        vbox4 = QVBoxLayout()
        groupbox.setLayout(vbox4)


        # Button for Sensor Measurement
        button_sensor1 = QPushButton("Sensor_Measurement")
        button_sensor1.clicked.connect(self.power_measurement)
        button_sensor1.setToolTip("This button is used for Taking Sensor measurements")
        vbox4.addWidget(button_sensor1)
        
        self.value_list = []
        button_measure_button = QPushButton("Point calculation")
        button_measure_button.clicked.connect(self.on_measure_button_clicked)
        button_measure_button.setToolTip("This button is used for taking points")
        vbox4.addWidget(button_measure_button)

        
        button_reference_measure = QPushButton("Reference_Measurement")
        button_reference_measure.clicked.connect(self.reference_measurement)
        button_reference_measure.setToolTip("This button is used for Reference measurements")
        vbox4.addWidget(button_reference_measure)

        button_Effeciency_measure = QPushButton("Effeciency Measurement")
        button_Effeciency_measure.clicked.connect(self.scan_area)
        button_Effeciency_measure.setToolTip("This button is used for Effeciency measurements")
        vbox4.addWidget(button_Effeciency_measure)

        
        self.labelreference = QLabel('Reference Ration: 0', self)
        self.label1 = QLabel('Sensor 1 Value: 0', self)
        self.label2 = QLabel('Sensor 2 Value: 0', self)
        vbox4.addWidget(self.labelreference)
        vbox4.addWidget(self.label1)
        vbox4.addWidget(self.label2)
        
        self.labelWavelength = QLabel('Wavelength (nm):', self)
        vbox4.addWidget(self.labelWavelength)
        self.lineeditWavelength = QLineEdit()
        self.lineeditWavelength.setPlaceholderText('Enter WaveLength and Press Enter')
        self.lineeditWavelength.setValidator(QDoubleValidator())
        self.lineeditWavelength.returnPressed.connect(self.return_pressed_wavelentgth)
        vbox4.addWidget(self.lineeditWavelength)

        self.labelaperture = QLabel('Aperture:', self)
        vbox4.addWidget(self.labelaperture)
        self.lineeditaperture = QLineEdit()
        self.lineeditaperture.setPlaceholderText('Enter aperture and Press Enter')
        self.lineeditaperture.setValidator(QDoubleValidator())
        self.lineeditaperture.returnPressed.connect(self.return_pressed_aperture)
        vbox4.addWidget(self.lineeditaperture)

        

     # function for Stepsize Returning
    def return_pressed_x(self):
        self.stepSize_x = float(self.lineedit.text())
        return self.stepSize_x

    
    def XAxisForward(self):
        l = []
        ss = self.stepSize_x
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.

            pidevice.ConnectUSB(serialnum='0115500179')
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):

                l.append(positions[axis])
                print(l)
            incrementedList = [z + ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            pitools.waitontarget(pidevice)

            positions = pidevice.qPOS('1')
            position = []
            for axis in pidevice.qPOS('1'):
                print('updated position of axis {} = {:.2f}'.format('X-axis', positions[axis]))
                position.append(positions[axis])

            strPositons = str(position)
            self.Xposition = strPositons
            self.labelX.setText(f'X-Axis Value: {self.Xposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)

    
    def XAxisBackward(self):

        l = []
        ss = self.stepSize_x
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.

            pidevice.ConnectUSB(serialnum='0115500179')
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):

                l.append(positions[axis])
                print(l)
            incrementedList = [z - ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            pitools.waitontarget(pidevice)

            positions = pidevice.qPOS('1')
            position = []
            for axis in pidevice.qPOS('1'):
                print('updated position of axis {} = {:.2f}'.format('X-axis', positions[axis]))
                position.append(positions[axis])

            strPositons = str(position)
            self.Xposition = strPositons
            self.labelX.setText(f'X-Axis Value: {self.Xposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)


     # function for Stepsize Returning
    def return_pressed_y(self):
        self.stepSize_y = float(self.lineedit1.text())
        return self.stepSize_y
    def YAxisForward(self):
        
        l = []
        ss = self.stepSize_y
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.DD']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.
            
            pidevice.ConnectUSB(serialnum='0135500452')
            pidevice.SVO('1',1)
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):

                l.append(positions[axis])
                print(l)
            incrementedList = [z + ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            pitools.waitontarget(pidevice)

            positions = pidevice.qPOS('1')
            position = []
            for axis in pidevice.qPOS('1'):
                print('updated position of axis {} = {:.2f}'.format('Y-axis', positions[axis]))
                position.append(positions[axis])

            strPositons = str(position)
            self.Yposition = strPositons
            self.labelY.setText(f'Y-Axis Value: {self.Yposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)
        
    def YAxisBackward(self):

        l = []
        ss = self.stepSize_y
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.DD']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.
            
            pidevice.ConnectUSB(serialnum='0135500452')
            pidevice.SVO('1',1)
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):

                l.append(positions[axis])
                print(l)
            incrementedList = [z - ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            pitools.waitontarget(pidevice)

            positions = pidevice.qPOS('1')
            position = []
            for axis in pidevice.qPOS('1'):
                print('updated position of axis {} = {:.2f}'.format('Y-axis', positions[axis]))
                position.append(positions[axis])

            strPositons = str(position)
            self.Yposition = strPositons
            self.labelY.setText(f'Y-Axis Value: {self.Yposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)


     # function for Stepsize Returning
    def return_pressed_z(self):
        self.stepSize_z = float(self.lineedit2.text())
        return self.stepSize_z
    
    def ZAxisForward(self):
        l = []
        ss = self.stepSize_z
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.

            pidevice.ConnectUSB(serialnum='0095500784')
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):
                
                l.append(positions[axis])
                print(l)
            incrementedList = [z + ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            pitools.waitontarget(pidevice)

            positions = pidevice.qPOS('1')
            position = []
            for axis in pidevice.qPOS('1'):
                print('updated position of axis {} = {:.2f}'.format('Z-axis', positions[axis]))
                position.append(positions[axis])
                
            strPositons = str(position)
            self.Zposition = strPositons
            self.labelZ.setText(f'Z-Axis Value: {self.Zposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)
    def ZAxisBackward(self):

        l = []
        ss = self.stepSize_z
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.

            pidevice.ConnectUSB(serialnum='0095500784')
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):

                l.append(positions[axis])
                print(l)
            incrementedList = [z - ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            pitools.waitontarget(pidevice)

            positions = pidevice.qPOS('1')
            position = []
            for axis in pidevice.qPOS('1'):
                print('updated position of axis {} = {:.2f}'.format('Z-axis', positions[axis]))
                position.append(positions[axis])

            strPositons = str(position)
            self.Zposition = strPositons
            self.labelZ.setText(f'Z-Axis Value: {self.Zposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)


    def power_measurement(self):
        rm = pyvisa.ResourceManager()
        # List all connected resources
        print("Resources detected\n{}\n".format(rm.list_resources()))
        print("Opening connection...")
        instrument = rm.open_resource('USB0::0x1313::0x8022::M00533936::INSTR', timeout=5000)
        
        print("Sending command...")
        command1 = ':POW1:VAL?'
        self.power_value1 = instrument.query(command1)
        command2 = ':POW2:VAL?'
        self.power_value2 = instrument.query(command2)
        
        print(f"Sensor 1: {self.power_value1}")
        print(f"Sensor 2: {self.power_value2}")
        
        print("Closing connection...")
        instrument.close()
        self.label1.setText(f'Sensor_1 Value: {self.power_value1}')
        self.label2.setText(f'Sensor_2 Value: {self.power_value2}')
        return self.power_value1, self.power_value2
        
    def reference_measurement(self):
        self.reference = float(self.power_value1)/float(self.power_value2)
        self.labelreference.setText(f'Refernece Value: {self.reference}')
        return self.reference


    # function for Stepsize Returning
    def return_pressed_wavelentgth(self):
        self.wavelength = float(self.lineeditWavelength.text())
        self.wavelength = self.wavelength * (1E-9)
        return self.wavelength

    def on_measure_button_clicked(self):
        
        measurement = {
            'measurement_index': self.measurement_index,
            'X-axis': self.Xposition,
            'Y-axis': self.Yposition,
            'Z-axis': self.Zposition,
            'Ch1': self.power_value1,
            'Ch2': self.power_value2,
            'Aperture': self.aperture
        }
        self.value_list.append(measurement)
        print("Measurement appended to the list:", measurement)

        print("list:", self.value_list)

        # Increment measurement index for the next measurement
        self.measurement_index += 1

        with open('AOI.pkl', 'wb') as file:
            pickle.dump(self.value_list, file)

    # function for Stepsize Returning
    def return_pressed_aperture(self):
        self.aperture = float(self.lineeditaperture.text())
        return self.aperture

    def scan_area(self):
        
        effeciency_list = []
        # list1 = loaded_list # for testing
        list1 = self.value_list # for actual
        x1 = list1[0]['X-axis']
        x1 = float(x1.strip('[]'))
        x2 = list1[1]['X-axis']
        x2 = float(x2.strip('[]'))
        print(x2)
        y1 = list1[0]['Y-axis']
        y1 = float(y1.strip('[]'))
        y4 = list1[2]['Y-axis']
        y4 = float(y4.strip('[]'))
        x_step = list1[0]['Aperture']
        x_step = float(x_step.strip('[]'))  # Define the step size for x-axis
        y_step = list1[0]['Aperture']
        y_step = float(y_step.strip('[]'))
    # x_step = 0.01 # Define the step size for x-axis
    # y_step = 0.01
        def XAxisForward_scan(ss):
            l = []
            ss = x_step
            # print('step Size', ss)
            CONTROLLERNAME = 'C-863'  # 'C-884' will also work
            STAGES = ['M-521.PD1']
            REFMODE = ('REF')
            with GCSDevice(CONTROLLERNAME) as pidevice:
            # InterfaceSetupDlg() is an interactive dialog. There are other methods to
            # connect to an interface without user interaction.
            
                pidevice.ConnectUSB(serialnum='0115500179')
                positions = pidevice.qPOS('1')
                for axis in pidevice.qPOS('1'):
            
                    l.append(positions[axis])
                    print(l)
                incrementedList = [z + ss for z in l]
                print(incrementedList)
                rangemax = [ '%.5f' % elem for elem in incrementedList ]
            
            
                pidevice.MOV('1', rangemax)
            
                pitools.waitontarget(pidevice)
            

        def YAxisBackward_scan(ss):
        
        
            l = []
            ss = x_step
            CONTROLLERNAME = 'C-863'  # 'C-884' will also work
            STAGES = ['M-521.DD']
            REFMODE = ('REF')
            with GCSDevice(CONTROLLERNAME) as pidevice:
            # InterfaceSetupDlg() is an interactive dialog. There are other methods to
            # connect to an interface without user interaction.
                
                pidevice.ConnectUSB(serialnum='0135500452')
                pidevice.SVO('1',1)
                positions = pidevice.qPOS('1')
                for axis in pidevice.qPOS('1'):
        
                    l.append(positions[axis])
                    print(l)
                incrementedList = [z - ss for z in l]
                print(incrementedList)
                rangemax = [ '%.5f' % elem for elem in incrementedList ]
        
        
                pidevice.MOV('1', rangemax)
        
                pitools.waitontarget(pidevice)
    

        def XAxisBackward_scan():
            
            
    
            l = []
            ss = (x2-x1)
            # print('step Size', ss)
            CONTROLLERNAME = 'C-863'  # 'C-884' will also work
            STAGES = ['M-521.PD1']
            REFMODE = ('REF')
            with GCSDevice(CONTROLLERNAME) as pidevice:
            # InterfaceSetupDlg() is an interactive dialog. There are other methods to
            # connect to an interface without user interaction.
    
                pidevice.ConnectUSB(serialnum='0115500179')
                positions = pidevice.qPOS('1')
                for axis in pidevice.qPOS('1'):
    
                    l.append(positions[axis])
                    print(l)
                incrementedList = [z - ss for z in l]
                print(incrementedList)
                rangemax = [ '%.5f' % elem for elem in incrementedList ]
    
    
                pidevice.MOV('1', rangemax)
    
                pitools.waitontarget(pidevice)
    
    
        for y in np.arange(y1, y4+y_step,y_step):
            for x in np.arange(x1, x2+x_step,x_step):
                a,b = power_measurement(self)
                measurement = {
                'X-axis': x,
                'Y-axis': y,
                'Ch1': a,
                'Ch2': b
            }
                effeciency_list.append(measurement)  # Return to starting y-position
                XAxisForward_scan(x+x_step)  # Move to next x-position
            XAxisBackward_scan()
            y += y_step
            with open('effeciency_measurements.pkl', 'wb') as file:
                pickle.dump(effeciency_list, file)
        

app = QApplication(sys.argv)
screen = Window()
screen.show()
try:
    sys.exit(app.exec_())
except SystemExit:    
    print('Good Luck!')      

step Size 0
[138.8999]
[138.8999]
updated position of axis X-axis = 138.90
step Size 0
[37.6357]
[37.6357]
updated position of axis Y-axis = 37.64
step Size 0
[87.6016]
[87.6016]
updated position of axis Z-axis = 87.60
Resources detected
('USB0::0x1313::0x8022::M00533936::INSTR', 'ASRL4::INSTR', 'ASRL5::INSTR', 'ASRL8::INSTR')

Opening connection...
Sending command...
Sensor 1: 1.08997865E-05

Sensor 2: 3.86848251E-06

Closing connection...
Measurement appended to the list: {'measurement_index': 0, 'X-axis': '[138.8999]', 'Y-axis': '[37.6357]', 'Z-axis': '[87.6016]', 'Ch1': '1.08997865E-05\n', 'Ch2': '3.86848251E-06\n', 'Aperture': 1.0}
list: [{'measurement_index': 0, 'X-axis': '[138.8999]', 'Y-axis': '[37.6357]', 'Z-axis': '[87.6016]', 'Ch1': '1.08997865E-05\n', 'Ch2': '3.86848251E-06\n', 'Aperture': 1.0}]
step Size 5.0
[138.8999]
[143.8999]
updated position of axis X-axis = 143.90
Resources detected
('USB0::0x1313::0x8022::M00533936::INSTR', 'ASRL4::INSTR', 'ASRL5::INSTR', 'ASRL8::IN

AttributeError: 'float' object has no attribute 'strip'

Good Luck!


In [ ]:
def scan_area():
    effeciency_list = []
    # list = loaded_list # for testing
    list1 = self.value_list # for actual
    x1 = list1[0]['X-axis']
    x1 = float(x1.strip('[]'))
    x2 = list1[1]['X-axis']
    x2 = float(x2.strip('[]'))
    print(x2)
    y1 = list1[0]['Y-axis']
    y1 = float(y1.strip('[]'))
    y4 = list1[2]['Y-axis']
    y4 = float(y4.strip('[]'))
    x_step = list1[0]['Aperture']
    x_step = float(x_step.strip('[]'))  # Define the step size for x-axis
    y_step = list1[0]['Aperture']
    y_step = float(y_step.strip('[]'))
    # x_step = 0.01 # Define the step size for x-axis
    # y_step = 0.01
    def XAxisForward_scan(ss):
        l = []
        ss = x_step
        # print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.
        
            pidevice.ConnectUSB(serialnum='0115500179')
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):
        
                l.append(positions[axis])
                print(l)
            incrementedList = [z + ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]
        
        
            pidevice.MOV('1', rangemax)
        
            pitools.waitontarget(pidevice)
        

    def YAxisBackward_scan(ss):
    
    
        l = []
        ss = x_step
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.DD']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.
            
            pidevice.ConnectUSB(serialnum='0135500452')
            pidevice.SVO('1',1)
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):
    
                l.append(positions[axis])
                print(l)
            incrementedList = [z - ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]
    
    
            pidevice.MOV('1', rangemax)
    
            pitools.waitontarget(pidevice)
    

    def XAxisBackward():

        l = []
        ss = (x2-x1)
        # print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.

            pidevice.ConnectUSB(serialnum='0115500179')
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):

                l.append(positions[axis])
                print(l)
            incrementedList = [z - ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            pitools.waitontarget(pidevice)

            positions = pidevice.qPOS('1')
            position = []
            for axis in pidevice.qPOS('1'):
                print('updated position of axis {} = {:.2f}'.format('X-axis', positions[axis]))
                position.append(positions[axis])

            strPositons_x = str(position)
            return strPositons_x

    for y in np.arange(y1, y4+y_step,y_step):
        for x in np.arange(x1, x2+x_step,x_step):
            a,b = measure_power()
            measurement = {
            'X-axis': x,
            'Y-axis': y,
            'Ch1': a,
            'Ch2': b
        }
            effeciency_list.append(measurement)  # Return to starting y-position
            XAxisForward_scan(x+x_step)  # Move to next x-position
        XAxisBackward(x1)
        y += y_step
    return effeciency_list
def frange(start, stop, step):
    # Floating-point range generator function
    while start < stop:
        yield start
        start += step


In [ ]:
import pickle
import numpy as np
with open('my_list.pkl', 'rb') as file:
    loaded_list = pickle.load(file)
loaded_list

In [ ]:
def frange(start, stop, step):
    while start < stop:
        yield start
        start += step
        start = round(start, 10) 

def move_x_actuator(x):
    # Placeholder function. Replace with actual code to move x-axis actuator.
    pass

def move_y_actuator(y):
    # Placeholder function. Replace with actual code to move y-axis actuator.
    pass

def measure_power(x, y):
    # Placeholder function. Replace with actual code to measure power.
    # This function should return the power reading at position (x, y).
    return 0.0  # Example dummy value

def perform_grid_scan(x1, y1, x2, y2, x3, y3, x4, y4, aperture_size):
    x_step = aperture_size  # Define the step size for x-axis
    y_step = aperture_size  # Define the step size for y-axis

    power_measurements = []

    for y in frange(y1, y4+y_step, y_step):  # Adjust range for y-axis
        for x in frange(x1, x2+x_step, x_step):  # Adjust range for x-axis
            power = measure_power(x, y)
            power_measurements.append({'x': x, 'y': y, 'power': power})

            move_x_actuator(x + x_step)
        
        move_x_actuator(x1)  # Return to starting x-position
        y += y_step  # Move to next y-position

    return power_measurements

# Example Usage
x1, y1 = 134.1, 67.6177
x2, y2 = 133.9, 67.6177
x3, y3 = 133.9, 72.6177
x4, y4 = 134.1, 72.6177

aperture_size = 0.1  # Aperture size in micrometers

power_measurements = perform_grid_scan(x1, y1, x2, y2, x3, y3, x4, y4, aperture_size)
print(power_measurements)
